In [236]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [237]:
sizeOfSets = 500
val = [None] * 4000 # Keep value (sizeOfSets * no. of categories)
sizeOfTweetContent = 75

In [238]:
categories = ['airlines', 'celebrities', 'colleges', 'fastfood', 
              'leagues', 'news', 'streamingplatforms', 'techgiants']

In [239]:
def inputDataInDataframe():
    overCount= 0
    for i in categories:
        direc:str = '/Users/sameernawaz/AnacondaProjects/Datasets/tweets/'+i+'/trainingTextClassification.csv'
        with open(direc, 'r') as csv_file:
            csv_reader = csv.reader(csv_file)
            count = 0
            for line in csv_reader:
                if len(line[6]) > sizeOfTweetContent:
                    val[overCount] = line
                    count += 1
                    overCount += 1
                    if count >= sizeOfSets :
                        break

In [240]:
inputDataInDataframe()

In [241]:
df = pd.DataFrame(val, columns = ['date', 'username', 'to', 'replies', 
                                  'retweets', 'favorites', 'text', 'geo', 
                                  'mentions', 'hashtags', 'id', 'permalink']) 

In [242]:
# Once data added in dataframe, Shuffle it
df = df.sample(frac=1).reset_index(drop=True)
# Add a unique id (rec_id) for all tweets
df = df.assign(rec_id=np.arange(len(df))).reset_index(drop=True)

In [243]:
df = df[['rec_id', 'username', 'text']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4000 entries, 0 to 3999
Data columns (total 3 columns):
rec_id      4000 non-null int64
username    4000 non-null object
text        4000 non-null object
dtypes: int64(1), object(2)
memory usage: 125.0+ KB


In [244]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

4000

In [245]:
# Extract TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(4000, 10013)

In [246]:
# Compute Pairwise Document Similarity

from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999
0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
1,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.011699,0.00000
2,0.0,0.0,1.0,0.0,0.0,0.032637,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.00000
3,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.011887,0.018572,0.0,...,0.0,0.0,0.0,0.0,0.074035,0.0,0.0,0.0,0.026500,0.00000
4,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.54168


In [257]:
# Get list of post IDs

posts_list = df['rec_id'].values
posts_list, posts_list.shape

(array([   0,    1,    2, ..., 3997, 3998, 3999]), (4000,))

In [262]:
posts_idx = np.where(posts_list == 3997)[0][0]
posts_idx

3997

In [263]:
posts_similarities = doc_sim_df.iloc[posts_idx].values
posts_similarities

array([0., 0., 0., ..., 1., 0., 0.])

In [264]:
# Get top 5 similar post IDs

similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
similar_posts_idxs

array([3997, 3707, 3543,  551, 2176])

In [251]:
similar_posts = posts_list[similar_movie_idxs]
similar_posts

array([3285, 3716, 3694, 1054,  263])

In [265]:
def relevant_posts(postId, posts = posts_list, doc_sims = doc_sim_df):
    # find post id
    posts_idx = np.where(posts == postId)[0][0]
    # get posts similarities
    posts_similarities = doc_sims.iloc[posts_idx].values
    # get top 5 similar post IDs
    similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
    # get top 5 posts
    similar_posts = posts[similar_posts_idxs]
    # return the top 5 posts
    return similar_posts

In [283]:
post_id = 3265

print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n')
for i in relevant_posts(postId = post_id, posts = posts_list, doc_sims = doc_sim_df):
    print("=>",(df.loc[df['rec_id'] == i]['text']).values[0])

Lawsuit filed in federal court accuses Education Dept. and America's federal consumer watchdog agency of abandoning its authority to oversee companies that service federal student loans.


=> NEW: Top Democratic senators revealed a new online privacy bill Tuesday that could mark a milestone in the lengthy push for a federal privacy law and would strengthen the Federal Trade Commission’s ability to enforce digital privacy protections.
=> A collaborative research project on estuaries led by the @GrahamInstitute and @UMSEAS has been awarded a $20M federal agreement to study the effects of climate change and land use on 29 coastal reserves. http://myumi.ch/7ZZ9X
=> A team of @UMengineering researchers will lead a federal, multi-institute project in developing synthetic neurons—the first challenge in creating synthetic cells—and re-creating the communication process that occurs throughout the body’s nervous system. http://myumi.ch/88Grr
=> As the federal government aims to implement new act